In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from dtaidistance import dtw, dtw_c, dtw_ndim
from numpy.random import choice, shuffle
from torch import nn
from sklearn.model_selection import train_test_split

In [2]:
%load_ext autoreload
%autoreload 1

from preprocessing import *
from models import Encoder, Decoder, Sequence2Sequence

%aimport preprocessing
%aimport models

# Example of multidimensional time-series

In [3]:
# !mkdir -p data
# !wget https://www.cs.ucr.edu/%7Eeamonn/time_series_data_2018/UCRArchive_2018.zip ../data
# !unzip data/UCRArchive_2018.zip -d ../data

In [4]:
# data_path = "../data/UCRArchive_2018/Earthquakes/Earthquakes_TRAIN.tsv"

In [5]:
# data = pd.read_csv(data_path, header=None, delimiter="\t").values

# # to remove Nan (only first columns)
# data = data[:, ~np.isnan(data).any(0)]

# X = data[:, 1:]
# y = data[:, 0].astype(np.short)

# Accelerometer dataset 

- 1: Working at Computer
- 2: Standing Up, Walking and Going up\down stairs
- 3: Standing
- 4: Walking
- 5: Going Up\Down Stairs
- 6: Walking and Talking with Someone
- 7: Talking while Standing

In [6]:
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00287/Activity%20Recognition%20from%20Single%20Chest-Mounted%20Accelerometer.zip -O ../data/Accelerometer.zip
# ! unzip ../data/Accelerometer.zip -d ./data

In [7]:
data_path = "../data/Activity Recognition from Single Chest-Mounted Accelerometer/1.csv"
CHANNELS = ['0', '1', '2']
LABELS = [1, 4, 5, 6]
LENGTH = 50
start_ident = 200

In [8]:
data = pd.read_csv(data_path,
                   names=['0', '1', '2', "labels"],
                   index_col=0,
                   dtype=np.float)

In [9]:
np.random.seed(0)

dataset = {}
for label in LABELS:
    X = get_class_timeseries(label, data, start_ident)
    X = np.linalg.norm(X, axis=1)[np.newaxis].T
    dataset[label] = slice_timeseries(X, LENGTH, count=45)
    

X = [x for _, ts in dataset.items() for x in ts]
y = [np.repeat(label, ts.shape[0]) for label, ts in dataset.items() for x in ts]
X = np.hstack(X).T
y = np.hstack(y).T

idxs = list(range(X.shape[0]))
shuffle(idxs)

X = X[idxs]
y = y[idxs]

In [10]:
w = 4
k = 4

input_dim = 2*k
hidden_dim = 5
n_layers = 3
batch_size = 180

In [11]:
X = [split_to_sequence(x, k, w) for x in X[choice(len(X), batch_size)]]

Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points are not considered.
Last 2 points 

In [14]:
X_train, X_test = train_test_split(X, test_size=0.5)

In [15]:
device = torch.device("cpu")

In [16]:
X_train = torch.tensor(X_train, dtype=torch.float).permute(1, 0, 2).to(device)
X_test = torch.tensor(X_test, dtype=torch.float).permute(1, 0, 2).to(device)

In [17]:
batch_size = 90

In [19]:
he = torch.randn(n_layers, batch_size, hidden_dim, dtype=torch.float).to(device)
ce = torch.randn(n_layers, batch_size, hidden_dim, dtype=torch.float).to(device)

hd = torch.randn(n_layers, batch_size, input_dim, dtype=torch.float).to(device)
cd = torch.randn(n_layers, batch_size, input_dim, dtype=torch.float).to(device)

In [20]:
from models import Encoder, Decoder, Sequence2Sequence
enc = Encoder(input_dim, hidden_dim, 1, 1, 3)
dec = Decoder(hidden_dim, input_dim, 1, 1, 3)

In [21]:
model = Sequence2Sequence(enc, dec)
model.to(device)

Sequence2Sequence(
  (encoder): Encoder(
    (lstm): LSTM(8, 5, num_layers=3)
  )
  (decoder): Decoder(
    (lstm): LSTM(5, 8, num_layers=3)
  )
)

In [167]:
loss_fn = nn.MSELoss()


In [168]:
N_STEP = 1000

def train(model, batch):
    model.train()
    for step in range(N_STEP):
        out = model(batch, he, ce, hd, cd)
        loss = loss_fn(batch, out)
        
        if step % 100 == 0:
            print("Loss:", loss)
        
        loss.backward()
        optim.step()
        
def valid(model, batch):
    model.eval()
    out = model(batch, he, ce, hd, cd)
    loss = loss_fn(batch, out)
    
    print("Valid loss:", loss)

In [169]:
valid(model, X_test)
train(model, X_train)
valid(model, X_test)


Valid loss: tensor(13639868., grad_fn=<MeanBackward0>)
Loss: tensor(13599683., grad_fn=<MeanBackward0>)
Loss: tensor(13592343., grad_fn=<MeanBackward0>)
Loss: tensor(13592343., grad_fn=<MeanBackward0>)
Loss: tensor(13592341., grad_fn=<MeanBackward0>)
Loss: tensor(13592342., grad_fn=<MeanBackward0>)
Loss: tensor(13592341., grad_fn=<MeanBackward0>)
Loss: tensor(13592357., grad_fn=<MeanBackward0>)
Loss: tensor(13592362., grad_fn=<MeanBackward0>)
Loss: tensor(13592356., grad_fn=<MeanBackward0>)
Loss: tensor(13592356., grad_fn=<MeanBackward0>)
Valid loss: tensor(13632520., grad_fn=<MeanBackward0>)


In [ ]:
from sklearn.model_selection import train_test_split

In [170]:
optim = torch.optim.Adam(model.parameters(), 0.05)
valid(model, X_test)
train(model, X_train)
valid(model, X_test)

valid(model, X_test)
train(model, X_train)
valid(model, X_test)

valid(model, X_test)
train(model, X_train)
valid(model, X_test)

valid(model, X_test)
train(model, X_train)
valid(model, X_test)
optim = torch.optim.Adam(model.parameters(), 0.01)


valid(model, X_test)
train(model, X_train)
valid(model, X_test)
optim = torch.optim.Adam(model.parameters())


valid(model, X_test)
train(model, X_train)
valid(model, X_test)

valid(model, X_test)
train(model, X_train)
valid(model, X_test)

valid(model, X_test)
train(model, X_train)
valid(model, X_test)

valid(model, X_test)
train(model, X_train)
valid(model, X_test)



Valid loss: tensor(13632520., grad_fn=<MeanBackward0>)
Loss: tensor(13592351., grad_fn=<MeanBackward0>)
Loss: tensor(13592351., grad_fn=<MeanBackward0>)
Loss: tensor(13592351., grad_fn=<MeanBackward0>)
Loss: tensor(13592352., grad_fn=<MeanBackward0>)
Loss: tensor(13592354., grad_fn=<MeanBackward0>)
Loss: tensor(13592354., grad_fn=<MeanBackward0>)
Loss: tensor(13592354., grad_fn=<MeanBackward0>)
Loss: tensor(13592354., grad_fn=<MeanBackward0>)
Loss: tensor(13592354., grad_fn=<MeanBackward0>)
Loss: tensor(13592354., grad_fn=<MeanBackward0>)
Valid loss: tensor(13632522., grad_fn=<MeanBackward0>)
Valid loss: tensor(13632522., grad_fn=<MeanBackward0>)
Loss: tensor(13592353., grad_fn=<MeanBackward0>)
Loss: tensor(13592353., grad_fn=<MeanBackward0>)
Loss: tensor(13592354., grad_fn=<MeanBackward0>)
Loss: tensor(13592360., grad_fn=<MeanBackward0>)
Loss: tensor(13592362., grad_fn=<MeanBackward0>)
Loss: tensor(13592362., grad_fn=<MeanBackward0>)
Loss: tensor(13592364., grad_fn=<MeanBackward0>)
Lo